# First Lab

### Importing modules

In [35]:
import os
import cv2 as cv
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch.cuda.amp import autocast, GradScaler
from google.colab.patches import cv2_imshow

### Creating Dataset Class to Model

In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, path: str):
        super().__init__()
        self.path = path
        self.images_list = os.listdir(path)
        self.images_count = len(self.images_list)

    def __len__(self):
        return self.images_count

    def getImageIndex(self, index):
        path = os.path.join(self.path, self.images_list[index])
        id = 0 if self.images_list[index].split('.')[0] == 'cat' else 1
        return path, id

    def __getitem__(self, index):
        img = None
        sub_index = 0
        while img is None:
            path, id_class = self.getImageIndex(index + sub_index)
            sub_index+=1
            img = cv.imread(path, cv.IMREAD_COLOR)
            if img is not None:
              break
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        img = cv.resize(img, (64, 64), interpolation=cv.INTER_AREA)
        img = img.transpose((2, 0, 1))

        t_img = torch.from_numpy(img)
        t_id = torch.tensor(id_class)

        return {'img': t_img, 'label': t_id}

In [15]:
path = './drive/MyDrive/images_1/'

train_dataset = Dataset(path + 'train')
test_dataset = Dataset(path + 'test')

### Creating DataLoaders

In [30]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=16,
                                           num_workers=1,
                                           shuffle=True,
                                           drop_last=True)
test_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=16,
                                          num_workers=1,
                                          shuffle=True,
                                          drop_last=False)

In [17]:
model = tv.models.resnet34(num_classes=2)

### Creating Loss function, Optimizer and Metrics

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [19]:
def accuracy(pred, label):
    answer = F.softmax(pred.detach(), dim=1).numpy().argmax(1) == label.numpy().argmax(1)
    return answer.mean()

In [20]:
scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma = 0.6
)

### Using CUDA

In [21]:
torch.cuda.is_available()

True

In [22]:
device = 'cuda'
model = model.to(device)
loss_fn = loss_fn.to(device)
use_amp = True
scaler = torch.cuda.amp.GradScaler()
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Training

In [26]:
epochs = 10
loss_epochs_list = []
acc_epochs_list = []
for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm(train_loader)):
        img, label = sample['img'], sample['label']
        label = F.one_hot(label, 2).float()
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        with autocast(use_amp):
          pred = model(img)
          loss = loss_fn(pred, label)

        scaler.scale(loss).backward()
        loss_item = loss.item()
        loss_val += loss_item

        scaler.step(optimizer)
        scaler.update()

        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

        pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
    scheduler.step()
    loss_epochs_list += [loss_val/len(train_loader)]
    acc_epochs_list += [acc_val/len(train_loader)]
    print(loss_epochs_list[-1])
    print(acc_epochs_list[-1])

  0%|          | 0/750 [00:00<?, ?it/s]

0.6519663600921631
0.6344166666666666


  0%|          | 0/750 [00:00<?, ?it/s]

0.5371609425544739
0.7336666666666667


  0%|          | 0/750 [00:00<?, ?it/s]

...

0.11238165498773257
0.9583333333333334


  0%|          | 0/750 [00:00<?, ?it/s]

0.07040054741501808
0.9778333333333333


  0%|          | 0/750 [00:00<?, ?it/s]

0.045022136797507606
0.9850833333333333


### Testing

In [31]:
device = 'cpu'
model = model.to(device)
loss_fn = loss_fn.to(device)

In [32]:
loss_val = 0
acc_val = 0
for sample in (pbar := tqdm(test_loader)):
    with torch.no_grad():
        img, label = sample['img'], sample['label']

        label = F.one_hot(label, 2).float()
        pred = model(img)

        loss = loss_fn(pred, label)
        loss_item = loss.item()
        loss_val += loss_item

        acc_current = accuracy(pred, label)
        acc_val += acc_current

    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
print(loss_val/len(train_loader))
print(acc_val/len(train_loader))

  0%|          | 0/751 [00:00<?, ?it/s]

0.03508612340502441
0.992
